In [1]:
from matplotlib import pyplot as plt
import time
import datetime
from utils_laj import *
from data_processing import get_SensorData, my_pca, series_to_supervised, lstm_sampling
from sklearn.model_selection import train_test_split
import pandas as pd

today = datetime.date.today()

In [2]:
path_checkpoint = './save/save_lstm/lstm_2_layers'
input_path = './input_data'
file = os.path.join(input_path,'sensors.csv')
target_file = os.path.join(input_path,'coke_target.csv')

In [3]:
# Считываем данные из файлов, нормализуем и применяем метод главных комполнентов
X, y, submit_X, mean_y, std_y = get_SensorData(file, target_file, nc=10)


In [4]:
#evals = pca.explained_variance_ratio_
#evals_cs = evals.cumsum()
#print(evals, evals_cs)
# Кумулятивное variance_explained равно 93%

In [5]:
import xgboost

In [6]:
# Размерности датасета с известным целевым y и полного датасета
print("Data X: %s, y %s, submit_X %s" % (X.shape, y.shape, submit_X.shape))

Data X: (10400, 11), y (10400, 2), submit_X (13272, 11)


In [7]:
# выделим последние 25% в тестовый датасет
x1_train, x1_test, y1_train, y1_test = train_test_split(X, y, test_size=2872, random_state=42, shuffle=False)

# Формируем сэмплы для LSTM по 48 рядов внахлест с шагом 1
timesteps = 48
x_train, y_train = lstm_sampling(x1_train.iloc[:,1:len(list(X))], y=y1_train['target'], timesteps=timesteps)
x_test, y_test = lstm_sampling(x1_test.iloc[:,1:len(list(X))], y=y1_test['target'], timesteps=timesteps)
#yl = y['target'][timesteps:]

# формируем данные конкурса по аналогичному принципу
Xs, ys = lstm_sampling(submit_X.iloc[(X.shape[0]-timesteps):,1:len(list(submit_X))], timesteps=timesteps)

print("Data x_train, y_train: (%s,%s);  x_test, y_test: (%s,%s); Xs, ys: (%s,%s) " % (x_train.shape, y_train.shape, x_test.shape, y_test.shape, Xs.shape, ys))



Samples length 7480
Targets length 7480
Samples length 2824
Targets length 2824
Samples length 2872
Targets length 0
Data x_train, y_train: ((7480, 48, 10),(7480, 48));  x_test, y_test: ((2824, 48, 10),(2824, 48)); Xs, ys: ((2872, 48, 10),None) 


In [8]:
    Train = False
    Predict = True
    plot = False

In [9]:
    batch_size = 359  # Batch size
    shift = batch_size
    #if Train == False: batch_size = 1

    sequence_length = timesteps  # Number of steps
    learning_rate = 8*10e-5  # 0.0001
    epochs = 500
    ann_hidden = 16

    n_channels = x_train.shape[2]

    lstm_size = 48  # Number LSTM units
    num_layers = 2  # 2  # Number of layers

In [10]:
    X = tf.placeholder(tf.float32, [None, sequence_length, n_channels], name='inputs')
    Y = tf.placeholder(tf.float32, [None, sequence_length], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    learning_rate_ = tf.placeholder(tf.float32, name='learning_rate')
    is_train = tf.placeholder(dtype=tf.bool, shape=None, name="is_train")

    conv_last_layer = X

    shape = conv_last_layer.get_shape().as_list()
    print('My Conv Shape:',shape)
    CNN_flat = tf.reshape(conv_last_layer, [-1, shape[1] * shape[2]])

    dence_layer_1 = dense_layer(CNN_flat, size=sequence_length * n_channels, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_1")
    lstm_input = tf.reshape(dence_layer_1, [-1, sequence_length, n_channels])

    cell = get_RNNCell(['LSTM'] * num_layers, keep_prob=keep_prob, state_size=lstm_size)
    init_state = cell.zero_state(batch_size, tf.float32)
    #init_state = tf.Variable(cell.zero_states(batch_size, tf.float32), trainable=False)
    rnn_output, states = tf.nn.dynamic_rnn(cell, lstm_input, dtype=tf.float32, initial_state=init_state)
    #with tf.control_dependencies([state.assign(states)]):
    #    output = tf.identity(output)
    
    stacked_rnn_output = tf.reshape(rnn_output, [-1, lstm_size])  # change the form into a tensor

    dence_layer_2 = dense_layer(stacked_rnn_output, size=ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_2")
    
    dence_layer_3 = dense_layer(dence_layer_2, size=ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_2_2")

    output = dense_layer(dence_layer_3, size=1, activation_fn=None, batch_norm=False, phase=is_train, drop_out=False,
                         keep_prob=keep_prob,
                         scope="fc_3_output")

    prediction = tf.reshape(output, [-1])
    y_flat = tf.reshape(Y, [-1])

    h = prediction - y_flat

    cost_function = tf.reduce_sum(tf.square(h))
    RMSE = tf.sqrt(tf.reduce_mean(tf.square(h)))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost_function)

    saver = tf.train.Saver()
    
    training_generator = batch_generator(x_train, y_train, batch_size, sequence_length, online=True, online_shift=shift)
    testing_generator = batch_generator(x_test, y_test, batch_size, sequence_length, online=True, online_shift=shift)
    #print(len(list(training_generator)))

    if Train: model_summary(learning_rate=learning_rate, batch_size=batch_size, lstm_layers=num_layers,
                            lstm_layer_size=lstm_size, fc_layer_size=ann_hidden, sequence_length=sequence_length,
                            n_channels=n_channels, path_checkpoint=path_checkpoint, spacial_note='')
        


My Conv Shape: [None, 48, 10]


In [11]:
    with tf.Session() as session:
        tf.global_variables_initializer().run()

        if Train == True:
            saver.restore(session, path_checkpoint)
            print("Model restored from file: %s" % path_checkpoint)

            cost = []
            plot_x = []
            plot_y1 = []
            plot_y2 = []
            iter_train = int(x_train.shape[0]/shift)
            iter_test = int(x_test.shape[0]/shift)
            print("Training set MSE")
            print("No epoches: ", epochs, "No itr: ", iter_train)
            __start = time.time()
            for ep in range(epochs):
                rmse_train1 = []
                rmse_test1 = []
                
                for itr in range(iter_train):
                    ## training ##
                    batch_x, batch_y = next(training_generator)
                    session.run(optimizer,
                                feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.7, learning_rate_: learning_rate})
                    rmse = RMSE.eval(feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0, learning_rate_: learning_rate})
                    cost.append(rmse)
                    rmse_train1.append(rmse)

                rmse_train = np.mean(rmse_train1)
                
                y_pred = []
                for itr in range(iter_test):
                    x_test_batch, y_test_batch = next(testing_generator)
                    rmse = RMSE.eval(feed_dict={X: x_test_batch, Y: y_test_batch, keep_prob: 1.0, learning_rate_: learning_rate})
                    rmse_test1.append(rmse)
                
                rmse_test = np.mean(rmse_test1)
                
                #rmse_train = session.run(RMSE, feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
                rmse_test = session.run(RMSE, feed_dict={X: x_test_batch, Y: y_test_batch, keep_prob: 1.0})
                plot_x.append(ep)
                plot_y1.append(rmse_train)
                plot_y2.append(rmse_test)
                
                #print(ep)

                time_per_ep = (time.time() - __start)
                time_remaining = ((epochs - ep) * time_per_ep) / 3600
                print("LSTM", "epoch:", ep, "RMSE-train:", rmse_train, "RMSE-test", rmse_test, "lr", learning_rate,
                      "\ttime/epoch:", round(time_per_ep, 2), "\ttime_remaining: ",
                      int(time_remaining), " hr:", round((time_remaining % 1) * 60, 1), " min", "\ttime_stamp: ",
                      datetime.datetime.now().strftime("%Y.%m.%d-%H:%M:%S"))
                __start = time.time()

                if ep % 10 == 0 and ep != 0:
                    save_path = saver.save(session, path_checkpoint)
                    if os.path.exists(path_checkpoint + '.meta'):
                        print("Model saved to file: %s" % path_checkpoint)
                    else:
                        print("NOT SAVED!!!", path_checkpoint)

                if ep % 100 == 0 and ep != 0: 
                    learning_rate = learning_rate / 2
                    plt.plot(plot_x, plot_y1, 'bo', plot_x, plot_y2, 'go')
                    plt.show()

            save_path = saver.save(session, path_checkpoint)
            if os.path.exists(path_checkpoint + '.meta'):
                print("Model saved to file: %s" % path_checkpoint)
            else:
                print("NOT SAVED!!!", path_checkpoint)
            plt.plot(plot_x, plot_y1, 'bo', plot_x, plot_y2, 'go')
            plt.show()
        else:
            saver.restore(session, path_checkpoint)
            print("Model restored from file: %s" % path_checkpoint)
            if Predict == True:
                print("Prediction for submit...")
                x_predict = Xs
                y_predict = np.zeros((Xs.shape[0],Xs.shape[1]))

                predict_generator = batch_generator(x_predict, y_predict, batch_size, sequence_length,
                                                       online=True, online_shift=shift)

                full_prediction = []

                iteration = int(x_predict.shape[0] / shift)
                #print("iteration: %i, predgen %s" % (iteration, predict_generator))
                print("#of validation points:", x_predict.shape[0], "#datapoints covers from minibatch:",
                      batch_size * sequence_length, "iterations/epoch", iteration)

                for itr in range(iteration):
                    x_validate_batch, y_validate_batch = next(predict_generator)
                    #print (itr)
                    __y_pred = session.run(output, feed_dict={X: x_validate_batch, Y: y_validate_batch, keep_prob: 1.0})
                    
                    for i in range(batch_size):
                        full_prediction.append(__y_pred[i*sequence_length])
                    #print(__y_pred.shape)
                    
                full_prediction = np.array(full_prediction)
                full_prediction = full_prediction.ravel()
            


INFO:tensorflow:Restoring parameters from ./save/save_lstm/lstm_2_layers
Model restored from file: ./save/save_lstm/lstm_2_layers
Prediction for submit...
#of validation points: 2872 #datapoints covers from minibatch: 17232 iterations/epoch 8


In [12]:
x_validate_batch.shape

(359, 48, 10)

In [13]:
full_prediction

array([0.26357067, 0.2778803 , 0.2826444 , ..., 0.50174373, 0.50683486,
       0.5157224 ], dtype=float32)

In [14]:
full_prediction.shape

(2872,)

In [15]:
shft = x_train.shape[0]+x_test.shape[0]+2*sequence_length

In [16]:
y_submit = full_prediction*std_y+mean_y
sub_file = pd.DataFrame()
sub_file["timestamp"] = submit_X["timestamp"][-2872:]
sub_file["target"] = y_submit[-2872:]
sub_file.to_csv('submit2.csv', index=False)

In [17]:
# LB 0.4705